In [1]:
# !pipenv install tflite-support

In [2]:
import ipywidgets as widgets
import IPython.display as ipd

import time

import numpy as np

import demucs
import demucs.utils
import demucs.separate
import demucs.pretrained

from resampy import resample
import playsound
import sounddevice as sd

from tensorflow.keras.layers import Input, Reshape, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, Dropout, Permute, Flatten, Dense
from tensorflow.keras.models import Model

import torch


import matplotlib as mpl
import matplotlib.pyplot as plt


playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.
2021-09-22 18:01:13.780462: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-22 18:01:13.780512: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def build_and_load_model(model_capacity, filename):
    """
    Build the CNN model and load the weights
    Parameters
    ----------
    model_capacity : 'tiny', 'small', 'medium', 'large', or 'full'
        String specifying the model capacity, which determines the model's
        capacity multiplier to 4 (tiny), 8 (small), 16 (medium), 24 (large),
        or 32 (full). 'full' uses the model size specified in the paper,
        and the others use a reduced number of filters in each convolutional
        layer, resulting in a smaller model that is faster to evaluate at the
        cost of slightly reduced pitch estimation accuracy.
    Returns
    -------
    model : tensorflow.keras.models.Model
        The pre-trained keras model loaded in memory
    """

    capacity_multiplier = {
        'tiny': 4, 'small': 8, 'medium': 16, 'large': 24, 'full': 32
    }[model_capacity]

    layers = [1, 2, 3, 4, 5, 6]
    filters = [n * capacity_multiplier for n in [32, 4, 4, 4, 8, 16]]
    widths = [512, 64, 64, 64, 64, 64]
    strides = [(4, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]

    x = Input(shape=(1024,), name='input', dtype='float32')
    y = Reshape(target_shape=(1024, 1, 1), name='input-reshape')(x)

    for l, f, w, s in zip(layers, filters, widths, strides):
        y = Conv2D(f, (w, 1), strides=s, padding='same',
                   activation='relu', name="conv%d" % l)(y)
        y = BatchNormalization(name="conv%d-BN" % l)(y)
        y = MaxPool2D(pool_size=(2, 1), strides=None, padding='valid',
                      name="conv%d-maxpool" % l)(y)
        y = Dropout(0.25, name="conv%d-dropout" % l)(y)

    y = Permute((2, 1, 3), name="transpose")(y)
    y = Flatten(name="flatten")(y)
    y = Dense(360, activation='sigmoid', name="classifier")(y)

    model = Model(inputs=x, outputs=y)

    model.load_weights(filename)
    model.compile('adam', 'binary_crossentropy')

    return model

In [4]:
pitch_model = build_and_load_model("medium", "../models/model-medium.h5")

2021-09-22 18:01:15.303050: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-22 18:01:15.303082: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-22 18:01:15.303110: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gldsn-hw): /proc/driver/nvidia/version does not exist
2021-09-22 18:01:15.303391: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import tensorflow.lite

In [6]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(pitch_model)

In [7]:
tflite_model = converter.convert()

2021-09-22 18:01:16.930682: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpj54i_2ma/assets


2021-09-22 18:01:19.017304: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-22 18:01:19.017531: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-22 18:01:19.038531: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2994400000 Hz
2021-09-22 18:01:19.044223: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.024ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.

2021-09-22 18:01:19.398844: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:345] Ignored output_format.
2021-09-22 18:01:19.398892: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:348] Ignored drop_control_dependency.
2021-09-22 18:01:19.426382: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.

In [22]:
with open('../models/crepe-medium.tflite', 'wb') as f:
    f.write(tflite_model)

In [23]:
from tflite_support import metadata as tf_metadata
from tflite_support import metadata_schema_py_generated

In [24]:
model_meta = metadata_schema_py_generated.ModelMetadataT()

In [25]:
model_meta.name = "Crepe medium"
model_meta.description = "Pitch detection from wavfile"
model_meta.author = "https://github.com/marl/"
model_meta.license = ("MIT License, https://github.com/marl/crepe/blob/master/LICENSE")

In [26]:
input_meta = metadata_schema_py_generated.TensorMetadataT()

output_meta = metadata_schema_py_generated.TensorMetadataT()

In [36]:
input_meta.name = "waveform signal"
input_meta.description = (
    "Input waveform samples array is expected to have shape (n_samples, 1024). "
    "Every signal should by normalized using its own mean and std. "
)
input_meta.content = metadata_schema_py_generated.ContentT()

input_meta.content.contentProperties = metadata_schema_py_generated.AudioPropertiesT()

In [37]:
input_meta.content.contentProperties

In [38]:
input_meta.content.contentProperties

In [39]:
input_meta.content.contentProperties.channels = 1
input_meta.content.contentProperties.sampleRate = 16000

In [45]:
output_meta.name = "Pitch bins probs"
output_meta.description = (
    "Each pitch bin size is 20 cents. abilitOut shape is (n_samples, 360)"
    "For more info please check out the CREPE docs"
)

In [46]:
subgraph = metadata_schema_py_generated.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

In [47]:
from tflite_support import flatbuffers

In [48]:
meta_builder = flatbuffers.Builder(0)
meta_builder.Finish(
            model_meta.Pack(meta_builder),
            tf_metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = meta_builder.Output()

In [49]:
populator = tf_metadata.MetadataPopulator.with_model_file("../models/crepe-medium.tflite")
populator.load_metadata_buffer(metadata_buf)
populator.populate()

In [ ]:
# with open('../models/crepe-medium.tflite', 'wb') as f:
#     f.write(tflite_model)